<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C2/W2/ungraded_labs/C2_W2_Lab_1_cats_v_dogs_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: ImageDataGenerator, Validación y Aumento de datos

En este cuaderno vamos a avanzar en el tratamiento de imágenes reales, vamos a ver la validacion del modelo en cada paso en entrenamiento y finalmente vamos a mejorar el modelo con el aumento de imágenes.

Veamos cómo se puede hacer esto en las siguientes secciones.

In [ ]:
# Instale este paquete para utilizar la GPU de Colab para la formación
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

## Rendimiento de referencia

Empezarás con un modelo que es muy eficaz en el aprendizaje de `Gatos vs Perros` sin aumento de datos. Es similar a los modelos anteriores que ha utilizado. Observa que hay cuatro capas convolucionales con 32, 64, 128 y 128 convoluciones respectivamente. El código es básicamente el mismo del laboratorio anterior, por lo que no repasaremos los detalles paso a paso, ya que lo has visto antes.

Entrenarás sólo durante 20 épocas para ahorrar tiempo, pero puedes aumentar esta cantidad si lo deseas.

In [ ]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile

# Extract the archive
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Assign training and validation set directories
base_dir = 'tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Colocarás la creación del modelo dentro de una función para que puedas inicializar fácilmente uno nuevo cuando utilices el aumento de datos más adelante en este cuaderno.

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

def create_model():
  '''Creates a CNN with 4 convolutional layers'''
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=1e-4),
                metrics=['accuracy'])
  
  return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

In [ ]:
# Constant for epochs
EPOCHS = 20

# Create a new model
model = create_model()

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=EPOCHS,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

A continuación, visualizará la pérdida y la precisión con respecto al conjunto de entrenamiento y validación. De nuevo utilizarás una función de conveniencia para que pueda ser reutilizada más tarde. Esta función acepta un objeto [Historia](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History) que contiene los resultados del método `fit()` que ejecutó anteriormente.

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_acc(history):
  '''Plots the training and validation loss and accuracy from a history object'''
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training Loss')
  plt.plot(epochs, val_loss, 'b', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
# Plot training results
plot_loss_acc(history)

En los resultados anteriores, se puede ver que la precisión de entrenamiento es superior al 90%, y la precisión de validación está en el rango del 70%-80%. Este es un gran ejemplo de _sobreajuste_, lo que significa que puede hacerlo muy bien con imágenes que ha visto antes, pero no tan bien con imágenes que no ha visto.


## Aumento de datos

Un método sencillo para evitar el sobreajuste es aumentar un poco las imágenes. Si lo piensas, la mayoría de las imágenes de un gato son muy similares: las orejas están arriba, luego los ojos, luego la boca, etc. Cosas como la distancia entre los ojos y las orejas también serán siempre bastante similares. 

¿Qué pasa si retocas un poco las imágenes?  En eso consiste el aumento de la imagen. Y hay una API que lo hace fácil.

Echa un vistazo al [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) que has estado utilizando para reescalar la imagen. Hay otras propiedades en él que puedes usar para aumentar la imagen. 

```
# Updated to do image augmentation
train_datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
```

Estas son sólo algunas de las opciones disponibles. Vamos a repasarlas rápidamente:

* `rotation_range` es un valor en grados (0-180) dentro del cual se pueden rotar aleatoriamente las imágenes.
* `width_shift` and `height_shift` son rangos (como una fracción de la anchura o la altura total) dentro de los cuales traducir aleatoriamente las imágenes vertical u horizontalmente.
* `shear_range` es para aplicar aleatoriamente transformaciones de cizallamiento.
* `zoom_range` es para aplicar aleatoriamente el zoom dentro de las imágenes.
* `horizontal_flip` es para voltear aleatoriamente la mitad de las imágenes horizontalmente. Esto es relevante cuando no hay supuestos de asimetría horizontal (por ejemplo, imágenes del mundo real).
* `fill_mode` es la estrategia utilizada para rellenar los píxeles recién creados, que pueden aparecer después de una rotación o un cambio de anchura/altura.

Ejecute las siguientes celdas para ver el impacto en los resultados. El código es similar al de la línea de base, pero la definición de `train_datagen` se ha actualizado para utilizar los parámetros descritos anteriormente.


In [ ]:
# Create new model
model_for_aug = create_model()

# Este código ha cambiado. Ahora, en lugar de que el ImageGenerator sólo reescalar
# la imagen, también rotamos y hacemos otras operaciones
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

# Train the new model
history_with_aug = model_for_aug.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=EPOCHS,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

In [ ]:
# Plot the results of training with data augmentation
plot_loss_acc(history_with_aug)

Como se puede ver, la precisión del entrenamiento ha bajado en comparación con la línea de base. Esto es de esperar porque (como resultado del aumento de datos) hay más variedad en las imágenes, por lo que el modelo necesitará más ejecuciones para aprender de ellas. Lo bueno es que la precisión de la validación ya no se estanca y está más en línea con los resultados del entrenamiento. Esto significa que el modelo está funcionando mejor con los datos no vistos. 




## Predicción con el modelo

Ahora eche un vistazo a la ejecución de una predicción utilizando el modelo. Este código le permitirá elegir 1 o más archivos de su sistema de archivos, cargarlos y ejecutarlos a través del modelo, dando una indicación de si el objeto es un caballo o un humano.

In [ ]:
import numpy as np
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
 
    # predicting images
    path = '/content/' + fn
    img = tf.keras.utils.load_img(path, target_size=(150, 150))
    x = tf.keras.utils.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    if classes[0]>0.5:
      print(fn + " is a cat")
    else:
      print(fn + " is a dog")

## Resumen

Este ejercicio ha mostrado un sencillo truco para evitar el sobreajuste. Puede mejorar sus resultados de referencia simplemente ajustando las mismas imágenes que ya tiene. La clase `ImageDataGenerator` tiene parámetros incorporados para hacer precisamente eso. Intente modificar los valores un poco más en el `train_datagen` y vea qué resultados obtiene.